I recently came across the Pycaret and I'm amazed at how much you can do with only a few lines of code. I thought it would be a great opporunity to compare the results of different models, including a blended model and a stacked model.   

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%capture
!pip install pycaret[full]
from pycaret.classification import * 

Notebook Sections
1. [Setup](#Setup)
2. [Compare Models](#Compare-Models)
3. [Extra Trees Classifier LB Results: 0.94729](#Extra-Trees-Classifier-LB-Results:-0.94729)
4. [Random Forest LB Results: 0.94443](#Random-Forest-LB-Results:-0.94443)
5. [Linear Disciminant Analysis LB Results: 0.71391](#Linear-Disciminant-Analysis-LB-Results:-0.71391)
5. [Blended Model LB Results: 0.95296](#Blended-Model-LB-Results:-0.95296)
6. [Stacked Model LB Results: 0.96215](#Stacked-Model-LB-Results:-0.96215)

# Setup

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv', index_col='row_id')
test_df = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv', index_col='row_id')

In [ ]:
%%time
# reduce memory usage from https://www.kaggle.com/hasanbasriakcay/tps-feb22-pycaret-model-comparisons-0-97-lb

def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

train = reduce_mem_usage(df)
test = reduce_mem_usage(test_df)

In [ ]:
%%time
model_setup = setup(data=df, 
                    target='target',
                    train_size=0.6,
                    session_id=123,
                    use_gpu = True,
                    data_split_shuffle = True,
                    data_split_stratify=True,
                    silent=True)

# Compare Models

In [ ]:
%%time
#compare different models, only 2 folds to speed up time
compare_models(fold=2)

In [ ]:
%%time
# extratrees model and results
et_model = create_model('et', fold=3)

In [ ]:
%%time
tuned_et_model = tune_model(et_model, fold=3)

In [ ]:
%%time
predict_model(et_model)

In [ ]:
%%time
predict_model(tuned_et_model)

So interestingly the tuned model did much worse than the untuned model. 

I tested the tuned results for a couple other algorithms and they all came out worse than the original model. 

This seems to be a common theme for others using Pycaret. https://github.com/pycaret/pycaret/issues/234

Next, I want to compare Leader Board scores for different models

In [ ]:
#extratrees LB results using the original extra trees model
et_test_preds = predict_model(et_model, data=test_df)
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission = pd.DataFrame(list(zip(submission.row_id, et_test_preds.Label)),columns = ['row_id', 'target'])
submission.to_csv('submission_et.csv', index = False)
submission.head()

## Extra Trees Classifier LB Results: 0.94729

In [ ]:
%%time
# randomforest model and results
rf_model = create_model('rf', fold=3)

In [ ]:
#random forest LB results using the original rf model
rf_test_preds = predict_model(rf_model, data=test_df)
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission = pd.DataFrame(list(zip(submission.row_id, rf_test_preds.Label)),columns = ['row_id', 'target'])
submission.to_csv('submission_rf.csv', index = False)
submission.head()

## Random Forest LB Results: 0.94443

In [ ]:
%%time
# linear discriminant model and results
lda_model = create_model('lda', fold=3)

In [ ]:
#lda results using the original lda model
lda_test_preds = predict_model(lda_model, data=test_df)
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission = pd.DataFrame(list(zip(submission.row_id, lda_test_preds.Label)),columns = ['row_id', 'target'])
submission.to_csv('submission_lda.csv', index = False)
submission.head()

## Linear Disciminant Analysis LB Results: 0.71391

The models I ran on the unseen test dataset all performed similarly to what was predicted on the validation set (40% of the train dataset)

# Testing a Blended Model and a Stacked Model

In [ ]:
%%time
lightgbm_model = create_model('lightgbm', fold=3)
knn_model = create_model('knn', fold=3)

In [ ]:
blended_model = blend_models(estimator_list = [et_model, rf_model, lda_model, knn_model, lightgbm_model], method='hard')

In [ ]:
predict_model(blended_model)

In [ ]:
blend_test_preds = predict_model(blended_model, data=test_df)
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission = pd.DataFrame(list(zip(submission.row_id, blend_test_preds.Label)),columns = ['row_id', 'target'])
submission.to_csv('submission_blend.csv', index = False)
submission.head()

## Blended Model LB Results: 0.95296

In [ ]:
# stacking models
stacked_model = stack_models(estimator_list = [et_model, rf_model, lda_model, lightgbm_model, knn_model],
                       meta_model = et_model)

In [ ]:
predict_model(stacked_model)

In [ ]:
stack_test_preds = predict_model(stacked_model, data=test_df)
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission = pd.DataFrame(list(zip(submission.row_id, stack_test_preds.Label)),columns = ['row_id', 'target'])
submission.to_csv('submission_stack.csv', index = False)
submission.head()

## Stacked Model LB Results: 0.96215

The stacked model performed the best so it would be worth spending more time on the stacked model to try to improve performance there. 